# Загружаем необходимые пакеты и импортируем библиотеки

---

In [ ]:
# устанавливаем библиотеку implicit 
%pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 75.1 MB/s eta 0:00:00


In [ ]:
# загружаем библиотеки
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
import implicit
import tensorflow as tf
from sklearn import preprocessing

In [ ]:
# Проверяем что у нас работает GPU
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
#GPU count and name
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-5bec47ce-e57f-9c3b-f90b-a138de8b0250)


In [ ]:
#use this command to see GPU activity while doing Deep Learning tasks, for this command 'nvidia-smi' and for above one to work, go to 'Runtime > change runtime type > Hardware Accelerator > GPU'
!nvidia-smi

Sat Apr 22 07:50:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    26W /  70W |    391MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Подготовка и обработка данных

---

In [ ]:
# загружаем данные из источника
data = pd.read_parquet('train_mfti.parquet', engine='pyarrow')

In [ ]:
# создаем копию
df = data.copy()

In [ ]:
# посмотрим на данные 
df.head()

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy


In [ ]:
# и так тоже посмотрим
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292588 entries, 0 to 12292587
Data columns (total 6 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   event_date       object
 1   event_timestamp  int64 
 2   vacancy_id_      int64 
 3   cookie_id        object
 4   user_id          object
 5   event_type       object
dtypes: int64(2), object(4)
memory usage: 562.7+ MB


In [ ]:
# Дубликатов нет в данных - но на всякий случай
df = df.drop_duplicates()

In [ ]:
# удаляем колонки, которые не будут принимать участия в обучении
df.drop(columns=['event_date', 'event_timestamp', 'user_id'], axis=1, inplace=True)

In [ ]:
# для удобства немного выравняем данные
df = df[['cookie_id', 'vacancy_id_', 'event_type']]

In [ ]:
df.head()

,cookie_id,vacancy_id_,event_type
0,97990f1a021d4be19aa3f955b7eacab4,129850,show_vacancy
1,03bf8c511fa949c79845a5d81b09aa1d,108347,show_vacancy
2,03bf8c511fa949c79845a5d81b09aa1d,109069,show_vacancy
3,03bf8c511fa949c79845a5d81b09aa1d,171425,show_vacancy
4,03bf8c511fa949c79845a5d81b09aa1d,252384,show_vacancy


 ### Обработка данных первая исследовательская позиция (Маштабирование взаимодействий и их силы)
 ---

In [ ]:
# Вот тут мы посмотрим на количество определенного вида взаимодействий в данных
df['event_type'].value_counts()

show_vacancy              6180832
preview_click_vacancy     4758461
click_response             382828
click_contacts             276819
preview_click_response     190130
click_favorite             155472
preview_click_favorite     106622
preview_click_contacts     101231
click_phone                 78667
preview_click_phone         15927
Name: event_type, dtype: int64

* 1 show_vacancy - просмотр вакансии
* 2 preview_click_vacancy - клик по карточке вакансии
* **3 click_response - отклик со страницы вакансии**
* **4 preview_click_response - отклик с карточки вакансии**
* 5 click_favorite - добавление вакансии в избранное  со страницы вакансии
* 6 preview_click_favorite - добавление вакансии в избраное с карточки вакансии
* **7 click_contacts - клик на контакты со страницы вакансии**
* **8 preview_click_contacts - клик на контакты из карточки вакансии**
* **9 click_phone - клик на номер телефона, указанный в вакансии**
* **10 preview_click_phone - клик на номер телефона из карточки вакансии**

Жирным выделены искомые ""откликнется"" и ""позвонит"""
! Стоит обратить внимание на то, что позиции 3 и 4 по частоте идут выше чем позиции 5 и 6, которые нам в положительное взаимодействие не засчитываются. В свзи с этим необходимо проработать несколько вариантов маштабирования данных

In [ ]:
# проверим если закодировать значимость взаимодейстивий по их количеству в выборке - при прочих стандартных параметрах этот
# способ показал результат 0,041
df['eventStrength'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 2.0,
                                     'click_response': 3.0,
                                     'click_contacts': 4.0,
                                     'preview_click_response': 5.0,
                                     'click_favorite': 6.0,
                                     'preview_click_favorite': 7.0,
                                     'preview_click_contacts': 8.0,
                                     'click_phone': 9.0,
                                     'preview_click_phone': 10.0})

In [ ]:
# проверим правильно ли мы закодировали взаимодействия
df['eventStrength'].value_counts()

1.0     6180832
2.0     4758461
3.0      382828
4.0      276819
5.0      190130
6.0      155472
7.0      106622
8.0      101231
9.0       78667
10.0      15927
Name: eventStrength, dtype: int64

Далее несколько ячеек кода для различных вариаций оценки взаимодействия их стоит прогнать с базовыми значениями и посмотреть на результаты (возможны и другие предложения):

In [ ]:
# проверить потом - вот такая модель должна дать нам больший перекос в сторону искомых взаимодействий перед обычными
df['eventStrength'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 1.0,
                                     'click_favorite': 1.0,
                                     'preview_click_favorite': 1.0,
                                     'click_response': 10.0,
                                     'preview_click_response': 10.0,
                                     'click_contacts': 10.0,
                                     'preview_click_contacts': 10.0,
                                     'click_phone': 10.0,
                                     'preview_click_phone': 10.0})

In [ ]:
# Вот это разбиение по нашему заданию, то есть первые 4 отрицательные взаимодействия остальные положительные
df['eventStrength'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 2.0,
                                     'click_favorite': 3.0,
                                     'preview_click_favorite': 4.0,
                                     'click_response': 5.0,
                                     'click_contacts': 6.0,
                                     'preview_click_response': 7.0,
                                     'preview_click_contacts': 8.0,
                                     'click_phone': 9.0,
                                     'preview_click_phone': 10.0})

In [ ]:
# Вот это разбиение по нашему заданию, но незначимые мы закодируем нулями нужные единицами, но не знаю как это пойдет
# Нули в матрице могут восприниматься как отсутсвие взаимодействия, а это неправда
df['eventStrength'] = df['event_type'].map({'show_vacancy': 0.0,
                                     'preview_click_vacancy': 0.0,
                                     'click_favorite': 0.0,
                                     'preview_click_favorite': 0.0,
                                     'click_response': 1.0,
                                     'click_contacts': 1.0,
                                     'preview_click_response': 1.0,
                                     'preview_click_contacts': 1.0,
                                     'click_phone': 1.0,
                                     'preview_click_phone': 1.0})

In [ ]:
# Вот это разбиение по нашему заданию, но незначимые мы закодируем 1 нужные 2
df['eventStrength'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 1.0,
                                     'click_favorite': 1.0,
                                     'preview_click_favorite': 1.0,
                                     'click_response': 2.0,
                                     'click_contacts': 2.0,
                                     'preview_click_response': 2.0,
                                     'preview_click_contacts': 2.0,
                                     'click_phone': 2.0,
                                     'preview_click_phone': 2.0})

In [ ]:
# Вот это разбиение по нашему заданию, ну и последний вариант это вот так:
df['eventStrength'] = df['event_type'].map({'show_vacancy': 0.1,
                                     'preview_click_vacancy': 0.1,
                                     'click_favorite': 0.1,
                                     'preview_click_favorite': 0.1,
                                     'click_response': 1.0,
                                     'click_contacts': 1.0,
                                     'preview_click_response': 1.0,
                                     'preview_click_contacts': 1.0,
                                     'click_phone': 1.0,
                                     'preview_click_phone': 1.0})

Группировка данных:

Еще один point по которому стоит провести исследование. Группировка схлопывает взаимодействия и выдает результат в виде силы взаимодействия

---

In [ ]:
# сгруппируем сочетания ваканции и человека при этом суммируем историю из взаимодействия в числах, чем больще число
# тем больше интерес человека к вакансии - а значит больше и сила взаимодействия

grouped_df = df.groupby(['cookie_id', 'vacancy_id_']).sum('numeric_only').reset_index()

In [ ]:
# попробуем еще 2 варината группировки - по максимуму

#grouped_df = df.groupby(['cookie_id', 'vacancy_id_']).max('numeric_only').reset_index()

In [ ]:
# попробуем еще 2 варината группировки - по максимуму

# grouped_df = df.groupby(['cookie_id', 'vacancy_id_']).mean('numeric_only').reset_index()

In [ ]:
# с 12 000 000 + наши данные схлопнулись до 4 500 000 +
grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4678585 entries, 0 to 4678584
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   cookie_id      object 
 1   vacancy_id_    int64  
 2   eventStrength  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 107.1+ MB


In [ ]:
# посмотрим как они выглядят теперь, далее можно будет сравнить как выглядит одно и тоже сравнение в контексте
# других вариантов маштабирования силы взаимодействий 
grouped_df.sample(10)

,cookie_id,vacancy_id_,eventStrength
1497123,518d0e5102c840faae32e7752f4f8c8a,251440,3.0
1193186,41038cebcd234a98abb8f72181ba477c,123518,3.0
2243228,7a3fdebf27474128adc38284117b5f8d,134968,2.0
3319807,b537d6ffa804460084474376a9719486,259380,3.0
1799644,62271f47a70744afb584f1f8bbf307d9,179871,12.0
3810251,d011a7852f8d4fa190e3c5715b8c61ff,258441,1.0
3627240,c614a48bd60a4ca29af681fbfba7d97a,140813,3.0
841798,2dd611e5c31f462ea52412e64b6395e3,145961,3.0
4130590,e1abd9b66f5f433ea1113521f8330bd4,176141,3.0
2736797,94cc2af99b5646569a2f4056ead85970,135363,3.0


In [ ]:
# посмотрим на одну вакансию в ключе ее взаимодействия с пользователями
grouped_df[grouped_df['vacancy_id_'] == 210068].sort_values('eventStrength', ascending=False).head(10)

,cookie_id,vacancy_id_,eventStrength
4271237,e9647deb68d6489c8f23fd7ff250f18c,210068,33.0
562580,1e9223caa87c453da3a102f3020327f7,210068,27.0
3160724,ac5710432b184069b1a4c0296fe3e41a,210068,12.0
4590495,fb3eebb8cfc54a3085a07e7f47dbed4c,210068,10.0
1243846,43de234dae2a41f98f394d8e1e615528,210068,9.0
2413147,83572d892cbd4fffbeecc28577eac17a,210068,9.0
754056,2901a05b78a0471a9f5ad505ee70168d,210068,9.0
1089404,3b435bddcb2c4ab4bb27d22a7c8d488e,210068,9.0
312745,110a687ec500431289aa6116974ffc36,210068,9.0
3234089,b08259ccb54a4e9b8fe4c5bf2dc0d180,210068,7.0


In [ ]:
# посмотрим на одного пользователя в ключе его взаимодействия с вакансиями
grouped_df[grouped_df['cookie_id'] == 'ac5710432b184069b1a4c0296fe3e41a'].sort_values('eventStrength', ascending=False).head(10)

,cookie_id,vacancy_id_,eventStrength
3160729,ac5710432b184069b1a4c0296fe3e41a,221774,38.0
3160734,ac5710432b184069b1a4c0296fe3e41a,250491,23.0
3160709,ac5710432b184069b1a4c0296fe3e41a,114808,23.0
3160716,ac5710432b184069b1a4c0296fe3e41a,168822,23.0
3160712,ac5710432b184069b1a4c0296fe3e41a,136830,21.0
3160733,ac5710432b184069b1a4c0296fe3e41a,249034,20.0
3160723,ac5710432b184069b1a4c0296fe3e41a,195789,20.0
3160732,ac5710432b184069b1a4c0296fe3e41a,248398,20.0
3160730,ac5710432b184069b1a4c0296fe3e41a,239208,13.0
3160724,ac5710432b184069b1a4c0296fe3e41a,210068,12.0


Вот к этому блоку нужно подойти очень внимательно - это выделение категориальных и числовых столбцов, в первых прогонах мы обошлись вот таким вариантом, но можно будет посмотреть и выделение нумированных вакансий (у нас в матрице с 0 до 100 000 значений нет, а номера есть, но может этого и ничего страшного)

---

In [ ]:
grouped_df['cookie_id'] = grouped_df['cookie_id'].astype("category") # наименование вакансии в категорию
#grouped_df['vacancy_id_'] = grouped_df['vacancy_id_'].astype("category") # наименование вакансии в категорию

In [ ]:
grouped_df['cookie_id_num'] = grouped_df['cookie_id'].cat.codes # вот тут выделяем числовые значения
#grouped_df['vacancy_id_num'] = grouped_df['vacancy_id_'].cat.codes # и вот тут выделим

Продолжаем готовить данные для разряженных матриц

---

In [ ]:
#grouped_df = grouped_df[['cookie_id', 'cookie_id_num', 'vacancy_id_', 'vacancy_id_num', 'eventStrength']]

In [ ]:
grouped_df = grouped_df[['cookie_id', 'cookie_id_num', 'vacancy_id_', 'eventStrength']]

In [ ]:
#data = grouped_df[['cookie_id_num', 'vacancy_id_num', 'eventStrength']]

In [ ]:
# В этом варианте кода мы пробуем запускать в матрицу номера вакансий которые нам дали изначально, что бы с ними потом не мучатся
data = grouped_df[['cookie_id_num', 'vacancy_id_', 'eventStrength']]

Вот тут стоит подумать о нормализации данных и вообще нужна ли она? Это второй исследовательский блок давайте пробовавать разную нормализацию

In [ ]:
data['eventStrength_norm'] = (data['eventStrength'] - data['eventStrength'].mean()) / data['eventStrength'].std()

In [ ]:
data.head() # мне кажется минусы нам не нужны будут проверю другую

,cookie_id_num,vacancy_id_,eventStrength,eventStrength_norm
0,0,137659,3.0,-0.319517
1,0,153975,3.0,-0.319517
2,0,174953,9.0,0.771015
3,0,176171,3.0,-0.319517
4,0,182445,6.0,0.225749


In [ ]:
data['eventStrength_norm'] = (data['eventStrength'] - data['eventStrength'].min()) / (data['eventStrength'].max() - data['eventStrength'].min())

In [ ]:
data.head() # мне кажется минусы нам не нужны будут проверю другую
# Вот этот датафрейм пойдет в матрицы (в следующий прогон загоним с нормализацией)

,cookie_id_num,vacancy_id_,eventStrength,eventStrength_norm
0,0,137659,3.0,0.002714
1,0,153975,3.0,0.002714
2,0,174953,9.0,0.010855
3,0,176171,3.0,0.002714
4,0,182445,6.0,0.006784


In [ ]:
# а вот это остается нашим базовым информационным датафреймом
grouped_df.head(10)

,cookie_id,cookie_id_num,vacancy_id_,eventStrength
0,0000c4548c3944c08972bbdc1fa4eb85,0,137659,3.0
1,0000c4548c3944c08972bbdc1fa4eb85,0,153975,3.0
2,0000c4548c3944c08972bbdc1fa4eb85,0,174953,9.0
3,0000c4548c3944c08972bbdc1fa4eb85,0,176171,3.0
4,0000c4548c3944c08972bbdc1fa4eb85,0,182445,6.0
5,0000c4548c3944c08972bbdc1fa4eb85,0,187529,3.0
6,0000d7508334414ca792c5ff66eb8c14,1,106676,3.0
7,0000d7508334414ca792c5ff66eb8c14,1,108690,3.0
8,0000d7508334414ca792c5ff66eb8c14,1,115744,3.0
9,0000d7508334414ca792c5ff66eb8c14,1,169615,3.0


Вот в этом блоке мы должны соотнести наш цифровой номер cookie со строчным, для вывода ответов

---

Первый вариант работает на словарях

In [ ]:
# кодируем числами все наши cookie_id
codes = grouped_df.cookie_id.astype('category')
# создаем словарик в котором ключ это код а значение текстовое поле cookie_id
cookie_dic = dict(enumerate(codes.cat.categories))
print (cookie_dic[4])

00015019b0594984afc3fd9fa9557490


In [ ]:
# и создадим обратный словарик
cookie_to_code = dict(zip(cookie_dic.values(), cookie_dic.keys()))

In [ ]:
print(cookie_to_code['00015019b0594984afc3fd9fa9557490'])

4


Другой способ через dataFrame пока заморозим

In [ ]:
cookie_dic_df = grouped_df.drop(columns = ['vacancy_id_','eventStrength'])

In [ ]:
cookie_dic_df.drop_duplicates(inplace=True)

In [ ]:
# вот у нас и справочник потом будем его мержить
cookie_dic_df.head()

,cookie_id,cookie_id_num
0,0000c4548c3944c08972bbdc1fa4eb85,0
6,0000d7508334414ca792c5ff66eb8c14,1
18,0000e63f423542fe8090575b611cea4d,2
26,00013bf3e58244b48d4dbdf43543c30c,3
32,00015019b0594984afc3fd9fa9557490,4


In [ ]:
# проверим наш предыдущий метод и попробуем на cookie_dic_df крутить туда и сюда код в номер и обратно
cookie_dic_df['cookie_id_back'] = cookie_dic_df['cookie_id_num'].map(cookie_dic)

In [ ]:
cookie_dic_df['cookie_id_to_code'] = cookie_dic_df['cookie_id'].map(cookie_to_code)

In [ ]:
# проверка пройдена
cookie_dic_df.head()

,cookie_id,cookie_id_num,cookie_id_back,cookie_id_to_code
0,0000c4548c3944c08972bbdc1fa4eb85,0,0000c4548c3944c08972bbdc1fa4eb85,0
6,0000d7508334414ca792c5ff66eb8c14,1,0000d7508334414ca792c5ff66eb8c14,1
18,0000e63f423542fe8090575b611cea4d,2,0000e63f423542fe8090575b611cea4d,2
26,00013bf3e58244b48d4dbdf43543c30c,3,00013bf3e58244b48d4dbdf43543c30c,3
32,00015019b0594984afc3fd9fa9557490,4,00015019b0594984afc3fd9fa9557490,4


Создание разряженных матриц

---

In [ ]:
# вот это уже проверенный код но он работает на кодированных номерах и cookie и vacancy_id - мы его отставили в сторону

# item/user spare matrix
sparse_item_user = sparse.csr_matrix((grouped_df['eventStrength'].astype(float), (grouped_df['vacancy_id_num'], grouped_df['cookie_id_num'])))
# user/item spare matrix
sparse_user_item = sparse.csr_matrix((grouped_df['eventStrength'].astype(float), (grouped_df['cookie_id_num'], grouped_df['vacancy_id_num'])))

In [ ]:
# а вот эта ячейка экспериментальный, главный ее смысл понять что базовые данные vacancy_id_ работают, мы провели эксперимент
# и они работают сейчас это базовые матрицы

# item/user spare matrix
sparse_item_user = sparse.csr_matrix((data['eventStrength'].astype(float), (data['vacancy_id_'], data['cookie_id_num'])))
# user/item spare matrix
sparse_user_item = sparse.csr_matrix((data['eventStrength'].astype(float), (data['cookie_id_num'], data['vacancy_id_'])))

In [ ]:
# предыдущий код но с нормализацией

# item/user spare matrix
sparse_item_user = sparse.csr_matrix((data['eventStrength_norm'].astype(float), (data['vacancy_id_'], data['cookie_id_num'])))
# user/item spare matrix
sparse_user_item = sparse.csr_matrix((data['eventStrength_norm'].astype(float), (data['cookie_id_num'], data['vacancy_id_'])))

In [ ]:
# проверим размерности
sparse_item_user.shape
# стоит обратить внимание на то что размерность вакансий 260 168 - это потому что первых 100000 вакансий нет
# если бы мы кодировали - то размерность бы была 160 000 что то такое

(260168, 330180)

In [ ]:
# и тут проверим
sparse_user_item.shape

(330180, 260168)

In [ ]:
# ну и посмотрим что там внутри
print(sparse_user_item[0])

  (0, 137659)	3.0
  (0, 153975)	3.0
  (0, 174953)	9.0
  (0, 176171)	3.0
  (0, 182445)	6.0
  (0, 187529)	3.0


In [ ]:
# и тут тоже посмотрим
print(sparse_item_user[200001])

  (0, 3607)	3.0
  (0, 37610)	3.0
  (0, 61744)	6.0
  (0, 68283)	1.0
  (0, 79418)	3.0
  (0, 82207)	7.0
  (0, 235945)	3.0
  (0, 293518)	9.0
  (0, 295951)	7.0


# Обучение моделей
Вот это третий исследовательский этап, мы должны много! очень много экспериментировать с:
 - factors = ?,
 - regularization = ?,
 - iterations = ?

---

In [ ]:
# Инициируем наши модели 
model_item = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=50)

In [ ]:
model_user = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=50)

Четвертый исследовательский блок связан с параметром альфа - он закомментирован в коде ниже. В тех статьях что я успел прочитать это чуть ли не главнй парметр, непонятно почему конечно, возможно он увеличивает и акцентирует отличие силе взаимодействия?, но тогда стоит задуматься нужна ли нам вообще нормализация? (в статьях встречались значения альфа в 15 и 50, надо пробовать)

Обучаем первую модель вакансия-пользователь (но мы ее скорее всего не будем использовать)

In [ ]:
# Вот эти параметры я подсмотрел, они не обязательные но наверное их лучше потом настраивать
#alpha = 50
#item_user_data = (sparse_item_user * alpha).astype('double')

item_user_data = sparse_item_user
# обучим модель на разряженной матрице  item/user/confidence weights - по русски вакансия/пользователь/веса заинтересованности
model_item.fit(item_user_data)

  0%|          | 0/50 [00:00<?, ?it/s]

Обучаем первую модель пользователь - вакансия (вот ее в основном и принимают наши функции которые мы будем использовать)

In [ ]:
# Вот эти параметры я подсмотрел, они не обязательные но наверное их лучше потом настраивать
#alpha = 50
#user_item_data = (sparse_user_item * alpha).astype('double')

user_item_data = sparse_user_item
# train the model on a sparse matrix of user/item confidence weights
model_user.fit(user_item_data)

  0%|          | 0/50 [00:00<?, ?it/s]

# Пробуем извечь пользу. То есть переходим к предсказаниям

---
(ниже собраны базовые функции библиотеки implicit)

## Создаем рекомендации для человека

---

Функция : recommend( )

Рекомендует элементы для пользователя
Вычисляет N лучших рекомендаций для пользователя и возвращает список идентификаторов элементов, оценка.

Параметры:

- userid ( int ) — идентификатор пользователя для расчета рекомендаций для
- user_items ( csr_matrix ) — разреженная матрица формы (number_users, number_items). Это позволяет нам искать понравившиеся предметы и их веса для пользователя. Это используется для фильтрации элементов, которые уже понравились, из выходных данных, а также для потенциального вычисления лучших элементов для этого пользователя.
- N ( int , необязательный ) — количество возвращаемых результатов
- filter_already_liked_items ( bool , необязательный ) — если установлено значение true, не возвращать элементы, присутствующие в обучающем наборе, которые были оценены указанным пользователем.
- filter_items ( последовательность целых чисел , необязательный ) — список дополнительных идентификаторов элементов, которые нужно отфильтровать из вывода.
- recalculate_user ( bool , необязательный ) — когда true, не полагайтесь на сохраненное состояние пользователя и вместо этого пересчитывайте из переданных в user_items

Возвращает:	
Список кортежей (itemid, score)

Тип возврата:	
список

Получим список рекомендаций для одного пользователя

---

In [ ]:
# модель тренируется на sparse matrix of user/item/confidence weights - выполнено
# model.fit(user_item_data) - выполнено

# порекомендуем вакансии для одного пользователя из тех что он еще не видел
userid = 0
recommendations = model_user.recommend(userid, user_item_data[userid], N = 5 , filter_already_liked_items = True)
print(recommendations) # уже отсортировано

(array([116823, 182870, 169194, 180382, 207423], dtype=int32), array([0.46656507, 0.41793132, 0.37318325, 0.3208992 , 0.30675507],
      dtype=float32))


In [ ]:
# или так мы можем сразу превращать их в список
userid = 0
ids, scores = model_user.recommend(userid, user_item_data[userid], N=5, filter_already_liked_items=True)
print(list(ids))
print(scores)

[116823, 182870, 169194, 180382, 207423]
[0.46656507 0.41793132 0.37318325 0.3208992  0.30675507]


In [ ]:
# порекомендуем вакансии для одного пользователя из тех что он уже взаимодействовал
userid = 0
recommendations = model_user.recommend(userid, user_item_data[userid], N = 5, filter_already_liked_items = False)
print(recommendations) # уже отсортировано

(array([174953, 116823, 182870, 169194, 180382], dtype=int32), array([0.6994151 , 0.46656507, 0.41793132, 0.37318325, 0.3208992 ],
      dtype=float32))


Получим списко рекомендаций для списка пользователей

---

In [ ]:
# Такой код позволяет получать рекомендации для списка пользователей сразу!!!
userids = np.arange(1000) # вот тут у нас список пользователей
ids, scores = model_user.recommend(userids, user_item_data[userids], N=5, filter_already_liked_items=True)
ids, ids.shape

(array([[116823, 182870, 169194, 180382, 207423],
        [217683, 260154, 163708, 115589, 210229],
        [110792, 148714, 258441, 113707, 244077],
        ...,
        [230322, 230326, 127887, 133932, 182879],
        [253946, 253817, 208761, 122122, 104528],
        [164602, 260154, 110421, 116823, 164481]], dtype=int32),
 (1000, 5))

In [ ]:
userids = np.arange(1000) # вот тут у нас список пользователей
group_recommendations = model_user.recommend(userids, user_item_data[userids], N=5, filter_already_liked_items=True)


In [ ]:
# Вот тут нужно написать код который записывает данные в датафрейм
result = pd.DataFrame({'пользователи' : userids, 'рекомендации' : list(ids)})

In [ ]:
result.head()

,пользователи,рекомендации
0,0,"[116823, 182870, 169194, 180382, 207423]"
1,1,"[217683, 260154, 163708, 115589, 210229]"
2,2,"[110792, 148714, 258441, 113707, 244077]"
3,3,"[242145, 153469, 126489, 197991, 207356]"
4,4,"[113707, 153245, 148714, 149024, 116823]"


## Найдем похожие вакансии на вакансию №

---

Функция: similar_items()

Вычисляет список похожих элементов

Параметры:	
- itemid (int) – идентификатор строки элемента, для которого извлекаются похожие элементы
- N (int, необязательно) – количество похожих элементов для возврата.
- react_users (csr_matrix, необязательно) – разреженная матрица формы (number_items, number_users). Это позволяет нам просмотреть откликнувшихся пользователей и их веса для элемента.
- recallculate_item (bool, необязательно) – При значении true не полагайтесь на сохраненное состояние элемента и вместо этого пересчитывайте из переданного в react_users

ВОЗВРАТ:	
Список кортежей (itemid, оценка)

Возвращаемый тип:	
Список

In [ ]:
# Найти похожую вакансию на вакансию №
itemid = 100005
related = model_user.similar_items(itemid)
print(list(related[0])) # стоит обратить внимание на то, что первой выходит сама эта вакансия
print(list(related[0][1:]))

[100005, 188746, 221652, 189497, 180201, 201215, 181059, 223080, 143875, 189711]
[188746, 221652, 189497, 180201, 201215, 181059, 223080, 143875, 189711]


## Найдем похожего пользователя

Функция: similar_users( )

Вычисляет список похожих пользователей

Параметры:

- userid ( int ) — идентификатор строки пользователя для получения похожих пользователей для
- N ( int , необязательный ) — количество похожих пользователей, которых необходимо вернуть.

Возвращает:	
Список кортежей (userid, score)

Тип возврата:	
список

In [ ]:
userid = 100005
related_users = model_user.similar_users(userid)
print(list(related_users[0])) # стоит обратить внимание на то, что первой выходит этот самый пользователь
print(list(related_users[0][1:]))

[100005, 113260, 192092, 143338, 307425, 281362, 81256, 58263, 131196, 66705]
[113260, 192092, 143338, 307425, 281362, 81256, 58263, 131196, 66705]


## Ранжируем элементы для пользователя

---

Функция: rank_items()

Ранжирует заданные элементы для пользователя и возвращает отсортированный список элементов.

Параметры:

- userid (int) – идентификатор пользователя для вычисления рекомендаций
- user_items (csr_matrix) – разреженная матрица формы (number_users, number_items). Это позволяет нам (необязательно) пересчитывать пользовательские факторы (см. параметр reconculate_user) по мере необходимости
- selected_items (List of itemids) – (список идентификаторов элементов)
- recalculate_user (bool, необязательно) – При значении true не полагайтесь на сохраненное состояние пользователя и вместо этого пересчитывайте из переданных в user_items



Возвращаемый тип:	
Список

In [ ]:
userid = 100005
rank_elements = model_user.rank_items(userid, user_item_data, selected_items = [100001, 100002, 100003, 100004])
print(rank_elements) 


(array([100002, 100001, 100004, 100003]), array([4.0011927e-03, 4.6750274e-04, 3.4725224e-05, 1.9556945e-05],
      dtype=float32))


<ipython-input-82-0aa5fe56e486>:2: DeprecationWarning: rank_items is deprecated. Use recommend with the 'items' parameter instead
  rank_elements = model_user.rank_items(userid, user_item_data, selected_items = [100001, 100002, 100003, 100004])


# Переходим к испытаниям на нашей ситуации

---

Загружаем данные

In [ ]:
# Вот это данные от партнера, стоит отметить что там есть строки с 2 вакансиями и более 5
test = pd.read_parquet('test_public_mfti.parquet', engine='pyarrow')

In [ ]:
test.head()

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"


In [ ]:
# получаем для них столбец с номерными вариантами cookie как мы это закодировали ранее
test['cookie_id_num'] = test['cookie_id'].map(cookie_to_code)

In [ ]:
test.head()

,cookie_id,vacancy_id_,cookie_id_num
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812...",54
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348...",266
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065...",810
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]",1727
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]",2395


In [ ]:
# проверим перевод в коды на правильность
print(cookie_to_code['000cd76cd33f43d4a1ac1d16d10f8bf7'])
print(cookie_to_code['0034bc7f404341ba8412665453e7825a'])
print(cookie_to_code['00a6c5a64a274c55a836402bdeb3b2c4'])

54
266
810


In [ ]:
# получаем из датафрейма список пользователей, которых мы загрузим в модель
test_list = list(test['cookie_id_num'])
test_list_str = list(test['cookie_id'])
print(test_list[:5])
print(test_list_str[:5])

[54, 266, 810, 1727, 2395]
['000cd76cd33f43d4a1ac1d16d10f8bf7', '0034bc7f404341ba8412665453e7825a', '00a6c5a64a274c55a836402bdeb3b2c4', '015937a125b14e74bdff1cddc49f9172', '01de50c280794cec8804f16f45f847b7']


In [ ]:
# Ячейка поличения предсказаний, загружаем в модель полученный список кодов пользоватлей 
userids = test_list
ids, scores = model_user.recommend(userids, user_item_data[userids], N=5, filter_already_liked_items=True)

In [ ]:
# а вот тут мы делаем хитро в первую колонку записываем строковые имена пользователей, а в список рекомендаций полученные предсказания
test_prediction = pd.DataFrame({'cookie_id' : test_list_str, 'top_recommendations' : list(ids)})

In [ ]:
# посмотрим что получилось (выглядит красиво как и требуется)
test_prediction.head()

,cookie_id,top_recommendations
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[146630, 260154, 215620, 164698, 182870]"
1,0034bc7f404341ba8412665453e7825a,"[253678, 171332, 138634, 190928, 246285]"
2,00a6c5a64a274c55a836402bdeb3b2c4,"[227708, 113305, 242642, 109079, 148714]"
3,015937a125b14e74bdff1cddc49f9172,"[114583, 193331, 164588, 192565, 205606]"
4,01de50c280794cec8804f16f45f847b7,"[203404, 128278, 173337, 138634, 137702]"


In [ ]:
# вот эта наша функция получения метрики
def precision_n(predictions, df_test_list_top, k=5):
  #recommendations num:
    n_rec_k = len(df_test_list_top)*k
  #true reccomendations:
    matches=0
    for row in range(len(predictions)):
      row_match = set(predictions['top_recommendations'][row]).intersection(set(df_test_list_top['vacancy_id_'][row]))  
      matches += len(row_match)
  #Results:
    precisions = matches / n_rec_k if n_rec_k != 0 else 0
    return precisions

In [ ]:
# Ну и собственно вот результат
precision_n(test_prediction, test)

0.04196891191709844